In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
 


In [39]:
df = pd.read_csv(r"C:\Users\ritod\Downloads\ipl\IPL_Data.csv",encoding="latin1")
df.head()


,Name,Team,Url,Type,ValueinCR,Full Name,Born,Age,National Side,Batting Style,...,Maidens,RunsConceded,Wickets,Best,3s,5s,BowlingAVG,EconomyRate,S/R,Mtc
0,Mayank Agarwal,PBKS,https://sports.ndtv.com/cricket/players/1430-m...,Batsman,12.00,Mayank Anurag Agarwal,"February 16, 1991 Bangalore, Karnataka","31 Years, 0 Months, 28 Days",India,Right Handed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Liam Livingstone,PBKS,https://sports.ndtv.com/cricket/players/64363-...,All-Rounder,11.50,Liam Stephen Livingstone,"August 4, 1993 Barrow-in-Furness, Cumberland","28 Years, 7 Months, 11 Days",England,Right Handed,...,0.0,13.0,0.0,0/13 v MI,0.0,0.0,NaN,13.00,NaN,1.0
2,Kagiso Rabada,PBKS,https://sports.ndtv.com/cricket/players/64042-...,Bowler,9.25,Kagiso Rabada,"May 25, 1995 Johannesburg","26 Years, 9 Months, 22 Days",South Africa,Left Handed,...,2.0,1560.0,76.0,4/21 v RCB,4.0,0.0,20.52,8.21,15.0,50.0
3,Shahrukh Khan,PBKS,https://sports.ndtv.com/cricket/players/113433...,All-Rounder,9.00,Masood Shahrukh Khan,"May 27, 1995 Chennai, Tamil Nadu","26 Years, 9 Months, 20 Days",India,Right Handed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Shikhar Dhawan,PBKS,https://sports.ndtv.com/cricket/players/737-sh...,Batsman,8.25,Shikhar Dhawan,"December 5, 1985 Delhi","36 Years, 3 Months, 10 Days",India,Left Handed,...,0.0,66.0,4.0,1/7 v DC,0.0,0.0,16.50,8.25,12.0,6.0


In [40]:
df.drop(['Url','Age','Full Name', 'Born', 'Sport','HighestInnScore','Best'],axis=1,inplace=True)




In [41]:
df.isnull().sum()

Name               0
Team               0
Type               0
ValueinCR          0
National Side      6
Batting Style     10
Bowling           31
MatchPlayed       75
InningsBatted     75
NotOuts           75
RunsScored        84
100s              75
50s               75
4s                75
6s                75
BattingAVG        92
BattingS/R        75
CatchesTaken      92
StumpingsMade     92
Ducks             76
R/O               76
InningsBowled    115
Overs            115
Maidens          115
RunsConceded     115
Wickets          115
3s               115
5s               115
BowlingAVG       132
EconomyRate      115
S/R              132
Mtc              115
dtype: int64

In [42]:
le = preprocessing.LabelEncoder()
df['Team']= le.fit_transform(df['Team'])
df['Type']= le.fit_transform(df['Type'])
df['National Side']= le.fit_transform(df['Type'])
df['Batting Style']= le.fit_transform(df['Batting Style'])
df['Bowling']= le.fit_transform(df['Bowling'])


In [43]:
df.columns

Index(['Name', 'Team', 'Type', 'ValueinCR', 'National Side', 'Batting Style',
       'Bowling', 'MatchPlayed', 'InningsBatted', 'NotOuts', 'RunsScored',
       '100s', '50s', '4s', '6s', 'BattingAVG', 'BattingS/R', 'CatchesTaken',
       'StumpingsMade', 'Ducks', 'R/O', 'InningsBowled', 'Overs', 'Maidens',
       'RunsConceded', 'Wickets', '3s', '5s', 'BowlingAVG', 'EconomyRate',
       'S/R', 'Mtc'],
      dtype='object')

In [44]:
df

,Name,Team,Type,ValueinCR,National Side,Batting Style,Bowling,MatchPlayed,InningsBatted,NotOuts,...,Overs,Maidens,RunsConceded,Wickets,3s,5s,BowlingAVG,EconomyRate,S/R,Mtc
0,Mayank Agarwal,6,1,12.00,1,1,6,100.0,95.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Liam Livingstone,6,0,11.50,0,1,4,9.0,9.0,1.0,...,1.0,0.0,13.0,0.0,0.0,0.0,NaN,13.00,NaN,1.0
2,Kagiso Rabada,6,2,9.25,2,0,7,50.0,18.0,8.0,...,190.0,2.0,1560.0,76.0,4.0,0.0,20.52,8.21,15.00,50.0
3,Shahrukh Khan,6,0,9.00,0,1,6,11.0,10.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Shikhar Dhawan,6,1,8.25,1,0,6,192.0,191.0,25.0,...,8.0,0.0,66.0,4.0,0.0,0.0,16.50,8.25,12.00,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,Gurkeerat Singh Mann,2,0,0.50,0,1,6,41.0,32.0,8.0,...,13.0,0.0,97.0,5.0,0.0,0.0,19.40,7.46,15.60,6.0
233,Noor Ahmad,2,2,0.30,2,1,11,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,Darshan Nalkande,2,0,0.20,0,1,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,Sai Sudharsan,2,0,0.20,0,0,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df = df.fillna(0)

In [55]:
df

,Name,Team,Type,ValueinCR,National Side,Batting Style,Bowling,MatchPlayed,InningsBatted,NotOuts,...,Overs,Maidens,RunsConceded,Wickets,3s,5s,BowlingAVG,EconomyRate,S/R,Mtc
0,Mayank Agarwal,6,1,12.00,1,1,6,100.0,95.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
1,Liam Livingstone,6,0,11.50,0,1,4,9.0,9.0,1.0,...,1.0,0.0,13.0,0.0,0.0,0.0,0.00,13.00,0.00,1.0
2,Kagiso Rabada,6,2,9.25,2,0,7,50.0,18.0,8.0,...,190.0,2.0,1560.0,76.0,4.0,0.0,20.52,8.21,15.00,50.0
3,Shahrukh Khan,6,0,9.00,0,1,6,11.0,10.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
4,Shikhar Dhawan,6,1,8.25,1,0,6,192.0,191.0,25.0,...,8.0,0.0,66.0,4.0,0.0,0.0,16.50,8.25,12.00,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,Gurkeerat Singh Mann,2,0,0.50,0,1,6,41.0,32.0,8.0,...,13.0,0.0,97.0,5.0,0.0,0.0,19.40,7.46,15.60,6.0
233,Noor Ahmad,2,2,0.30,2,1,11,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
234,Darshan Nalkande,2,0,0.20,0,1,8,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
235,Sai Sudharsan,2,0,0.20,0,0,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           237 non-null    object 
 1   Team           237 non-null    int32  
 2   Type           237 non-null    int32  
 3   ValueinCR      237 non-null    float64
 4   National Side  237 non-null    int64  
 5   Batting Style  237 non-null    int32  
 6   Bowling        237 non-null    int32  
 7   MatchPlayed    237 non-null    float64
 8   InningsBatted  237 non-null    float64
 9   NotOuts        237 non-null    float64
 10  RunsScored     237 non-null    float64
 11  100s           237 non-null    float64
 12  50s            237 non-null    float64
 13  4s             237 non-null    float64
 14  6s             237 non-null    float64
 15  BattingAVG     237 non-null    float64
 16  BattingS/R     237 non-null    float64
 17  CatchesTaken   237 non-null    float64
 18  StumpingsM

In [57]:
df['Type']

0      1
1      0
2      2
3      0
4      1
      ..
232    0
233    2
234    0
235    0
236    0
Name: Type, Length: 237, dtype: int32

In [58]:
df.columns

Index(['Name', 'Team', 'Type', 'ValueinCR', 'National Side', 'Batting Style',
       'Bowling', 'MatchPlayed', 'InningsBatted', 'NotOuts', 'RunsScored',
       '100s', '50s', '4s', '6s', 'BattingAVG', 'BattingS/R', 'CatchesTaken',
       'StumpingsMade', 'Ducks', 'R/O', 'InningsBowled', 'Overs', 'Maidens',
       'RunsConceded', 'Wickets', '3s', '5s', 'BowlingAVG', 'EconomyRate',
       'S/R', 'Mtc'],
      dtype='object')

In [59]:
df_bats = pd.DataFrame(columns = ['Name', 'Team', 'Type', 'ValueinCR', 'National Side', 'Batting Style',
       'Bowling','MatchPlayed', 'InningsBatted', 'NotOuts',
       'RunsScored', '100s', '50s', '4s', '6s', 'BattingAVG', 'BattingS/R',
       'CatchesTaken', 'StumpingsMade', 'Ducks', 'R/O', 'InningsBowled',
       'Overs', 'Maidens', 'RunsConceded', 'Wickets', '3s', '5s',
       'BowlingAVG', 'EconomyRate', 'S/R', 'Mtc'])
j=0
for i in range(len(df['Type'])):
    if df['Type'][i] == 1:
        df_bats.loc[j]= df.iloc[i]
        j+=1

In [60]:
df_bats.head()

,Name,Team,Type,ValueinCR,National Side,Batting Style,Bowling,MatchPlayed,InningsBatted,NotOuts,...,Overs,Maidens,RunsConceded,Wickets,3s,5s,BowlingAVG,EconomyRate,S/R,Mtc
0,Mayank Agarwal,6,1,12.00,1,1,6,100.0,95.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,Shikhar Dhawan,6,1,8.25,1,0,6,192.0,191.0,25.0,...,8.0,0.0,66.0,4.0,0.0,0.0,16.5,8.25,12.0,6.0
2,Bhanuka Rajapaksa,6,1,0.50,1,2,13,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,Kane Williamson,9,1,14.00,1,1,6,63.0,62.0,15.0,...,3.0,0.0,31.0,0.0,0.0,0.0,0.0,10.33,0.0,2.0
4,Rahul Tripathi,9,1,8.50,1,1,9,62.0,60.0,7.0,...,1.0,0.0,12.0,0.0,0.0,0.0,0.0,12.00,0.0,1.0


In [64]:
df_bowl = pd.DataFrame(columns = ['Name', 'Team', 'Type', 'ValueinCR', 'National Side', 'Batting Style',
       'Bowling', 'MatchPlayed', 'InningsBatted', 'NotOuts',
       'RunsScored', '100s', '50s', '4s', '6s', 'BattingAVG', 'BattingS/R',
       'CatchesTaken', 'StumpingsMade', 'Ducks', 'R/O', 'InningsBowled',
       'Overs', 'Maidens', 'RunsConceded', 'Wickets', '3s', '5s',
       'BowlingAVG', 'EconomyRate', 'S/R', 'Mtc'])
j=0
for i in range(len(df['Type'])):
    if df['Type'][i] == 2:
        df_bowl.loc[j]= df.iloc[i]
        j+=1

df_bowl.head()

,Name,Team,Type,ValueinCR,National Side,Batting Style,Bowling,MatchPlayed,InningsBatted,NotOuts,...,Overs,Maidens,RunsConceded,Wickets,3s,5s,BowlingAVG,EconomyRate,S/R,Mtc
0,Kagiso Rabada,6,2,9.25,2,0,7,50.0,18.0,8.0,...,190.0,2.0,1560.0,76.0,4.0,0.0,20.52,8.21,15.00,50.0
1,Rahul Chahar,6,2,5.25,2,1,5,42.0,11.0,4.0,...,150.0,0.0,1117.0,43.0,3.0,0.0,25.97,7.44,20.93,41.0
2,Arshdeep Singh,6,2,4.00,2,0,3,23.0,3.0,2.0,...,76.1,1.0,669.0,30.0,3.0,1.0,22.30,8.78,15.23,23.0
3,Vaibhav Arora,6,2,2.00,2,1,8,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
4,Nathan Ellis,6,2,0.75,2,1,8,3.0,2.0,1.0,...,11.0,0.0,90.0,1.0,0.0,0.0,90.00,8.18,66.00,3.0


In [63]:
df_all = pd.DataFrame(columns = ['Name', 'Team', 'Type', 'ValueinCR', 'National Side', 'Batting Style',
       'Bowling','MatchPlayed', 'InningsBatted', 'NotOuts',
       'RunsScored', '100s', '50s', '4s', '6s', 'BattingAVG', 'BattingS/R',
       'CatchesTaken', 'StumpingsMade', 'Ducks', 'R/O', 'InningsBowled',
       'Overs', 'Maidens', 'RunsConceded', 'Wickets', '3s', '5s',
       'BowlingAVG', 'EconomyRate', 'S/R', 'Mtc'])
j=0
for i in range(len(df['Type'])):
    if df['Type'][i] == 0:
        df_all.loc[j]= df.iloc[i]
        j+=1

df_all.head()

,Name,Team,Type,ValueinCR,National Side,Batting Style,Bowling,MatchPlayed,InningsBatted,NotOuts,...,Overs,Maidens,RunsConceded,Wickets,3s,5s,BowlingAVG,EconomyRate,S/R,Mtc
0,Liam Livingstone,6,0,11.5,0,1,4,9.0,9.0,1.0,...,1.0,0.0,13.0,0.0,0.0,0.0,0.0,13.00,0.0,1.0
1,Shahrukh Khan,6,0,9.0,0,1,6,11.0,10.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,Odean Smith,6,0,6.0,0,1,9,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,Harpreet Brar,6,0,3.8,0,0,12,10.0,6.0,6.0,...,32.0,1.0,228.0,5.0,1.0,0.0,45.6,7.12,38.4,10.0
4,Raj Bawa,6,0,2.0,0,0,10,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
